In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from sklearn.model_selection import train_test_split
import os
import numpy as np

In [7]:
PS1A = pd.read_csv("PS1A_train.csv")
PS1A

,Index,C1,C2,C3,C4,C5,C6,C7,C8,C9,Target
0,1,3,9,9,0,6,0,1,0.60,0.43,342
1,2,3,7,11,0,0,0,1,0.84,0.53,243
2,3,1,1,23,0,4,1,2,0.40,0.82,91
3,4,3,9,6,0,0,0,1,0.58,0.88,20
4,5,2,5,9,0,6,0,1,0.54,0.49,218
...,...,...,...,...,...,...,...,...,...,...,...
6946,6947,2,4,9,0,5,1,1,0.46,0.88,359
6947,6948,2,5,17,0,5,1,1,0.66,0.34,812
6948,6949,3,8,12,0,3,1,1,0.80,0.43,189
6949,6950,1,2,7,0,2,1,1,0.24,0.65,100


In [8]:
PS1A = PS1A.iloc[:, 1:]
PS1A

,C1,C2,C3,C4,C5,C6,C7,C8,C9,Target
0,3,9,9,0,6,0,1,0.60,0.43,342
1,3,7,11,0,0,0,1,0.84,0.53,243
2,1,1,23,0,4,1,2,0.40,0.82,91
3,3,9,6,0,0,0,1,0.58,0.88,20
4,2,5,9,0,6,0,1,0.54,0.49,218
...,...,...,...,...,...,...,...,...,...,...
6946,2,4,9,0,5,1,1,0.46,0.88,359
6947,2,5,17,0,5,1,1,0.66,0.34,812
6948,3,8,12,0,3,1,1,0.80,0.43,189
6949,1,2,7,0,2,1,1,0.24,0.65,100


In [80]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Set the seed for reproducibility
torch.manual_seed(42)

# Define the hyperparameters
n_samples = 1000
n_features = 5
hidden_size_1 = 256
learning_rate = 0.01
epochs = 150
batch_size = 32

# Define the MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(9, hidden_size_1)
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_1)
        self.fc3 = nn.Linear(hidden_size_1, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = (self.fc3(x))
        return x

# Define the loss function
def loss_function(y_hat, y):
    MSE = F.mse_loss(y_hat, y, reduction='sum')
    return MSE

# Generate synthetic data
X = torch.tensor(np.array(PS1A.iloc[:, :9]))
X = X.to(torch.float32)
w_true = torch.randn(9, 1, dtype= torch.float32)
b_true = torch.randn(1, 1, dtype=torch.float32)
y = torch.mm(X, w_true) + b_true + torch.randn(len(X), 1, dtype=torch.float32)

# Split the data into training and validation sets
train_size = int(0.8 * n_samples)
val_size = n_samples - train_size
train_indices = torch.randperm(n_samples)[:train_size]
val_indices = torch.randperm(n_samples)[train_size:]
X_train = X[train_indices]
y_train = y[train_indices]
X_val = X[val_indices]
y_val = y[val_indices]

# Initialize the MLP and optimizer
mlp = MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate)

# Train the MLP
train_losses = []
val_losses = []
for epoch in range(epochs):
    train_loss = 0
    for batch_start in range(0, train_size, batch_size):
        batch_end = min(batch_start + batch_size, train_size)
        batch_X = X_train[batch_start:batch_end].to(torch.float32)
        batch_y = y_train[batch_start:batch_end]
        optimizer.zero_grad()
        y_hat = mlp(torch.tensor(batch_X))
        loss = loss_function(y_hat, batch_y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * (batch_end - batch_start)
    train_loss /= train_size
    train_losses.append(train_loss)
    with torch.no_grad():
        y_val_hat = mlp(X_val)
        val_loss = loss_function(y_val_hat, y_val)
        val_losses.append(val_loss.item())

    print('Epoch [{}/{}], Train Loss: {:.3f}, Validation Loss: {:.3f}'.format(epoch+1, epochs, train_loss, val_loss.item()))

# Generate synthetic data using the trained model
X_test = torch.randn(len(X), 9, dtype=torch.float32)
y_test = mlp(X_test).detach()

# Print the first few rows of the synthetic data
print('X1,  X2,  X3,  X4,  X5,  y')
for i in range(100):
    print(int(X_test[i, 1]), ' ',int(X_test[i, 2]),' ', int(X_test[i, 3]),' ', int(X_test[i, 4]),' ', int(X_test[i, 5]),' ', int(X_test[i, 6]), ' ',int(X_test[i, 7]),' ', int(X_test[i, 8]),' ',int(y_test[i]))


C:\Users\Sanje\AppData\Local\Temp\ipykernel_12336\4075742967.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_hat = mlp(torch.tensor(batch_X))


Epoch [1/150], Train Loss: 344.197, Validation Loss: 1368.165
Epoch [2/150], Train Loss: 129.490, Validation Loss: 625.765
Epoch [3/150], Train Loss: 94.940, Validation Loss: 493.836
Epoch [4/150], Train Loss: 72.280, Validation Loss: 421.576
Epoch [5/150], Train Loss: 59.933, Validation Loss: 371.599
Epoch [6/150], Train Loss: 49.426, Validation Loss: 345.667
Epoch [7/150], Train Loss: 46.475, Validation Loss: 329.510
Epoch [8/150], Train Loss: 43.391, Validation Loss: 267.382
Epoch [9/150], Train Loss: 42.523, Validation Loss: 255.053
Epoch [10/150], Train Loss: 40.062, Validation Loss: 234.290
Epoch [11/150], Train Loss: 38.057, Validation Loss: 229.854
Epoch [12/150], Train Loss: 36.983, Validation Loss: 208.381
Epoch [13/150], Train Loss: 35.554, Validation Loss: 200.423
Epoch [14/150], Train Loss: 33.959, Validation Loss: 186.536
Epoch [15/150], Train Loss: 32.917, Validation Loss: 192.349
Epoch [16/150], Train Loss: 32.874, Validation Loss: 190.770
Epoch [17/150], Train Loss: 32

Epoch [136/150], Train Loss: 14.627, Validation Loss: 166.422
Epoch [137/150], Train Loss: 15.338, Validation Loss: 144.540
Epoch [138/150], Train Loss: 16.541, Validation Loss: 130.380
Epoch [139/150], Train Loss: 15.615, Validation Loss: 125.369
Epoch [140/150], Train Loss: 13.806, Validation Loss: 117.781
Epoch [141/150], Train Loss: 12.916, Validation Loss: 146.741
Epoch [142/150], Train Loss: 13.091, Validation Loss: 139.991
Epoch [143/150], Train Loss: 13.751, Validation Loss: 154.659
Epoch [144/150], Train Loss: 13.954, Validation Loss: 122.262
Epoch [145/150], Train Loss: 13.903, Validation Loss: 129.365
Epoch [146/150], Train Loss: 13.176, Validation Loss: 133.594
Epoch [147/150], Train Loss: 13.134, Validation Loss: 143.746
Epoch [148/150], Train Loss: 12.453, Validation Loss: 128.899
Epoch [149/150], Train Loss: 12.417, Validation Loss: 126.538
Epoch [150/150], Train Loss: 13.134, Validation Loss: 123.303
X1,  X2,  X3,  X4,  X5,  y
0   -2   0   0   -1   0   0   0   0
1   0  

In [81]:
PS1B = pd.read_csv("PS1A_train.csv")

In [82]:
PS1B

,Index,C1,C2,C3,C4,C5,C6,C7,C8,C9,Target
0,1,3,9,9,0,6,0,1,0.60,0.43,342
1,2,3,7,11,0,0,0,1,0.84,0.53,243
2,3,1,1,23,0,4,1,2,0.40,0.82,91
3,4,3,9,6,0,0,0,1,0.58,0.88,20
4,5,2,5,9,0,6,0,1,0.54,0.49,218
...,...,...,...,...,...,...,...,...,...,...,...
6946,6947,2,4,9,0,5,1,1,0.46,0.88,359
6947,6948,2,5,17,0,5,1,1,0.66,0.34,812
6948,6949,3,8,12,0,3,1,1,0.80,0.43,189
6949,6950,1,2,7,0,2,1,1,0.24,0.65,100


In [85]:
PS1B = PS1B.iloc[:, 1:]

In [114]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Set the seed for reproducibility
torch.manual_seed(42)

# Define the hyperparameters
n_samples = 1000
n_features = 5
hidden_size_1 = 256
learning_rate = 0.01
epochs = 150
batch_size = 32

# Define the MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(9, hidden_size_1)
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_1)
        self.fc3 = nn.Linear(hidden_size_1, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = (self.fc3(x))
        return x

# Define the loss function
def loss_function(y_hat, y):
    MSE = F.mse_loss(y_hat, y, reduction='sum')
    return MSE

# Generate synthetic data
X = torch.tensor(np.array(PS1B.iloc[:, :9]))
X = X.to(torch.float32)
w_true = torch.randn(9, 1, dtype= torch.float32)
b_true = torch.randn(1, 1, dtype=torch.float32)
y = torch.mm(X, w_true) + b_true + torch.randn(len(X), 1, dtype=torch.float32)

# Split the data into training and validation sets
train_size = int(0.8 * n_samples)
val_size = n_samples - train_size
train_indices = torch.randperm(n_samples)[:train_size]
val_indices = torch.randperm(n_samples)[train_size:]
X_train = X[train_indices]
y_train = y[train_indices]
X_val = X[val_indices]
y_val = y[val_indices]

# Initialize the MLP and optimizer
mlp = MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate)

# Train the MLP
train_losses = []
val_losses = []
for epoch in range(epochs):
    train_loss = 0
    for batch_start in range(0, train_size, batch_size):
        batch_end = min(batch_start + batch_size, train_size)
        batch_X = X_train[batch_start:batch_end].to(torch.float32)
        batch_y = y_train[batch_start:batch_end]
        optimizer.zero_grad()
        y_hat = mlp(torch.tensor(batch_X))
        loss = loss_function(y_hat, batch_y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * (batch_end - batch_start)
    train_loss /= train_size
    train_losses.append(train_loss)
    with torch.no_grad():
        y_val_hat = mlp(X_val)
        val_loss = loss_function(y_val_hat, y_val)
        val_losses.append(val_loss.item())

    print('Epoch [{}/{}], Train Loss: {:.3f}, Validation Loss: {:.3f}'.format(epoch+1, epochs, train_loss, val_loss.item()))

# Generate synthetic data using the trained model
X_test = torch.randn(len(X), 9, dtype=torch.float32)
y_test = mlp(X_test).detach()

# Print the first few rows of the synthetic data
print('X1,  X2,  X3,  X4,  X5,  y')
for i in range(100):
    print(int(X_test[i, 1]), ' ',int(X_test[i, 2]),' ', int(X_test[i, 3]),' ', int(X_test[i, 4]),' ', int(X_test[i, 5]),' ', int(X_test[i, 6]), ' ',int(X_test[i, 7]),' ', int(X_test[i, 8]),' ',int(y_test[i]))


C:\Users\Sanje\AppData\Local\Temp\ipykernel_12336\3189199829.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_hat = mlp(torch.tensor(batch_X))


Epoch [1/150], Train Loss: 344.197, Validation Loss: 1368.165
Epoch [2/150], Train Loss: 129.490, Validation Loss: 625.765
Epoch [3/150], Train Loss: 94.940, Validation Loss: 493.836
Epoch [4/150], Train Loss: 72.280, Validation Loss: 421.576
Epoch [5/150], Train Loss: 59.933, Validation Loss: 371.599
Epoch [6/150], Train Loss: 49.426, Validation Loss: 345.667
Epoch [7/150], Train Loss: 46.475, Validation Loss: 329.510
Epoch [8/150], Train Loss: 43.391, Validation Loss: 267.382
Epoch [9/150], Train Loss: 42.523, Validation Loss: 255.053
Epoch [10/150], Train Loss: 40.062, Validation Loss: 234.290
Epoch [11/150], Train Loss: 38.057, Validation Loss: 229.854
Epoch [12/150], Train Loss: 36.983, Validation Loss: 208.381
Epoch [13/150], Train Loss: 35.554, Validation Loss: 200.423
Epoch [14/150], Train Loss: 33.959, Validation Loss: 186.536
Epoch [15/150], Train Loss: 32.917, Validation Loss: 192.349
Epoch [16/150], Train Loss: 32.874, Validation Loss: 190.770
Epoch [17/150], Train Loss: 32

Epoch [135/150], Train Loss: 15.758, Validation Loss: 150.533
Epoch [136/150], Train Loss: 14.627, Validation Loss: 166.422
Epoch [137/150], Train Loss: 15.338, Validation Loss: 144.540
Epoch [138/150], Train Loss: 16.541, Validation Loss: 130.380
Epoch [139/150], Train Loss: 15.615, Validation Loss: 125.369
Epoch [140/150], Train Loss: 13.806, Validation Loss: 117.781
Epoch [141/150], Train Loss: 12.916, Validation Loss: 146.741
Epoch [142/150], Train Loss: 13.091, Validation Loss: 139.991
Epoch [143/150], Train Loss: 13.751, Validation Loss: 154.659
Epoch [144/150], Train Loss: 13.954, Validation Loss: 122.262
Epoch [145/150], Train Loss: 13.903, Validation Loss: 129.365
Epoch [146/150], Train Loss: 13.176, Validation Loss: 133.594
Epoch [147/150], Train Loss: 13.134, Validation Loss: 143.746
Epoch [148/150], Train Loss: 12.453, Validation Loss: 128.899
Epoch [149/150], Train Loss: 12.417, Validation Loss: 126.538
Epoch [150/150], Train Loss: 13.134, Validation Loss: 123.303
X1,  X2,

In [115]:
PS1C = pd.read_csv("PS1C_train.csv")

In [92]:
PS1C = PS1C.iloc[:, 1:]
PS1C

,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,...,C21,C22,C23,C24,C25,C26,C27,C28,C29,Target
0,-0.287744,0.222853,-2.998039,-0.690275,-0.674866,-0.404163,0.405966,-2.319937,0.775369,-0.057652,...,0.031281,0.395767,-0.097444,-0.357012,-0.230728,-0.239019,0.272303,0.128075,29.89,0
1,-1.290936,1.081047,1.689179,1.857100,-0.633250,-0.808269,0.466850,-0.099381,0.161682,0.374637,...,0.233531,0.229839,0.367753,-0.345799,-0.614424,-0.322642,0.159902,0.221186,38.94,0
2,-0.939394,1.391180,-1.432628,-1.598677,0.829657,1.103927,-0.276995,0.600442,-0.757897,-0.567842,...,0.161042,0.918642,0.382627,-0.575711,0.240417,-0.217267,0.191545,-0.014056,399.40,0
3,0.170442,-1.945039,0.273686,0.740743,-0.530378,0.199080,-0.220303,0.476406,-0.388658,-1.035835,...,-0.399412,-1.022962,0.261353,-0.051819,-0.165381,0.207375,-0.060959,-0.035671,1.79,0
4,-1.362427,-0.280416,-1.437635,-1.381315,-0.249888,-1.454115,-0.136940,-0.081281,1.275863,1.595319,...,-0.496381,-0.743399,0.391387,-0.437921,-0.595208,-0.528237,0.009113,-0.047769,39.95,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.675362,-1.441520,-0.488052,-0.143098,-0.451366,-0.183787,-0.273587,-0.565989,0.796306,-1.167151,...,-0.605608,-1.179861,0.332523,0.413023,-0.270728,0.374114,-0.060189,-0.048870,40.47,0
996,0.653237,0.422112,2.385177,0.102150,-0.474317,0.335611,-0.123559,-0.851044,0.672238,-0.776345,...,-0.508938,-1.628332,0.217964,-0.063715,0.168564,-0.468109,-0.023746,0.026011,14.95,0
997,-0.322419,0.501951,1.033746,-0.317320,0.431012,-0.323396,0.054104,0.461689,-0.075649,-1.596593,...,0.101561,0.054401,-0.307052,-0.993344,0.495802,-0.291367,0.038254,0.051725,144.00,0
998,-0.330676,-0.372405,1.134086,-0.371052,0.236852,-0.676891,0.246508,0.766768,0.394456,0.669769,...,0.281381,0.803754,0.162364,0.663850,-0.158224,-0.611218,0.041129,-0.031165,29.00,0


In [126]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Set the seed for reproducibility
torch.manual_seed(42)

# Define the hyperparameters
n_samples = 1000
n_features = 5
hidden_size_1 = 256
hidden_size_2 = 256
learning_rate = 0.01
epochs = 5
batch_size = 32

# Define the MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(29, hidden_size_1)
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.fc3 = nn.Linear(hidden_size_2, 1)

        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x

# Define the loss function
def loss_function(y_hat, y):
    MSE = F.mse_loss(y_hat, y, reduction='sum')
    return MSE

# Generate synthetic data
X = torch.tensor(np.array(PS1C.iloc[:, :29]))
X = X.to(torch.float32)
w_true = torch.randn(29, 1, dtype= torch.float32)
b_true = torch.randn(1, 1, dtype=torch.float32)
y = torch.mm(X, w_true) + b_true + torch.randn(len(X), 1, dtype=torch.float32)

# Split the data into training and validation sets
train_size = int(0.8 * n_samples)
val_size = n_samples - train_size
train_indices = torch.randperm(n_samples)[:train_size]
val_indices = torch.randperm(n_samples)[train_size:]
X_train = X[train_indices]
y_train = y[train_indices]
X_val = X[val_indices]
y_val = y[val_indices]

# Initialize the MLP and optimizer
mlp = MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate)

# Train the MLP
train_losses = []
val_losses = []
for epoch in range(epochs):
    train_loss = 0
    for batch_start in range(0, train_size, batch_size):
        batch_end = min(batch_start + batch_size, train_size)
        batch_X = X_train[batch_start:batch_end].to(torch.float32)
        batch_y = y_train[batch_start:batch_end]
        optimizer.zero_grad()
        y_hat = mlp(torch.tensor(batch_X))
        loss = loss_function(y_hat, batch_y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * (batch_end - batch_start)
    train_loss /= train_size
    train_losses.append(train_loss)
    with torch.no_grad():
        y_val_hat = mlp(X_val)
        val_loss = loss_function(y_val_hat, y_val)
        val_losses.append(val_loss.item())

    print('Epoch [{}/{}], Train Loss: {:.3f}, Validation Loss: {:.3f}'.format(epoch+1, epochs, train_loss, val_loss.item()))

# Generate synthetic data using the trained model
X_test = torch.randn(len(X), 29, dtype=torch.float32)
y_test = mlp(X_test).detach()

# Print the first few rows of the synthetic data
print('X1,  X2,  X3,  X4,  X5,  y')
for i in range(5):
    for j in range(5):
        print(int(X_test[i, j]), ' ',  )
        torch.round([y_test[i]])


C:\Users\Sanje\AppData\Local\Temp\ipykernel_12336\3738104898.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_hat = mlp(torch.tensor(batch_X))


Epoch [1/5], Train Loss: 39521079.280, Validation Loss: 251578224.000
Epoch [2/5], Train Loss: 39521007.920, Validation Loss: 251578224.000
Epoch [3/5], Train Loss: 39521008.000, Validation Loss: 251578224.000
Epoch [4/5], Train Loss: 39521007.600, Validation Loss: 251578224.000
Epoch [5/5], Train Loss: 39521007.440, Validation Loss: 251578224.000
X1,  X2,  X3,  X4,  X5,  y
0  


TypeError: round(): argument 'input' (position 1) must be Tensor, not list

In [128]:
PS1D = pd.read_csv("PS1D_train.csv")

In [129]:
PS1D = PS1D.iloc[:, 1:]

In [130]:
PS1D

,C1,C2,C3,C4,C5,C6,C7,C8,Target
0,1,0.665,0.500,0.175,1.2975,0.6075,0.3140,0.3150,9
1,2,0.535,0.460,0.145,0.7875,0.3395,0.2005,0.2000,8
2,1,0.625,0.495,0.155,1.0485,0.4870,0.2120,0.3215,11
3,2,0.625,0.490,0.110,1.1360,0.5265,0.1915,0.2925,9
4,3,0.410,0.300,0.090,0.2800,0.1410,0.0575,0.0750,8
...,...,...,...,...,...,...,...,...,...
830,3,0.365,0.295,0.095,0.2500,0.1075,0.0545,0.0800,9
831,2,0.545,0.430,0.140,0.8320,0.4355,0.1700,0.2010,9
832,2,0.525,0.430,0.165,0.7170,0.2890,0.1745,0.1950,10
833,3,0.540,0.415,0.155,0.7020,0.3220,0.1670,0.1900,10


In [132]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Set the seed for reproducibility
torch.manual_seed(42)

# Define the hyperparameters
n_samples = 1000
n_features = 5
hidden_size_1 = 256
hidden_size_2 = 256
learning_rate = 0.01
epochs = 5
batch_size = 32

# Define the MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(8, hidden_size_1)
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.fc3 = nn.Linear(hidden_size_2, 1)

        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x

# Define the loss function
def loss_function(y_hat, y):
    MSE = F.mse_loss(y_hat, y, reduction='sum')
    return MSE

# Generate synthetic data
X = torch.tensor(np.array(PS1D.iloc[:, :8]))
X = X.to(torch.float32)
w_true = torch.randn(8, 1, dtype= torch.float32)
b_true = torch.randn(1, 1, dtype=torch.float32)
y = torch.mm(X, w_true) + b_true + torch.randn(len(X), 1, dtype=torch.float32)

# Split the data into training and validation sets
train_size = int(0.8 * len(X))
val_size = n_samples - train_size
train_indices = torch.randperm(n_samples)[:train_size]
val_indices = torch.randperm(n_samples)[train_size:]
X_train = X[train_indices]
y_train = y[train_indices]
X_val = X[val_indices]
y_val = y[val_indices]

# Initialize the MLP and optimizer
mlp = MLP()
optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate)

# Train the MLP
train_losses = []
val_losses = []
for epoch in range(epochs):
    train_loss = 0
    for batch_start in range(0, train_size, batch_size):
        batch_end = min(batch_start + batch_size, train_size)
        batch_X = X_train[batch_start:batch_end].to(torch.float32)
        batch_y = y_train[batch_start:batch_end]
        optimizer.zero_grad()
        y_hat = mlp(torch.tensor(batch_X))
        loss = loss_function(y_hat, batch_y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * (batch_end - batch_start)
    train_loss /= train_size
    train_losses.append(train_loss)
    with torch.no_grad():
        y_val_hat = mlp(X_val)
        val_loss = loss_function(y_val_hat, y_val)
        val_losses.append(val_loss.item())

    print('Epoch [{}/{}], Train Loss: {:.3f}, Validation Loss: {:.3f}'.format(epoch+1, epochs, train_loss, val_loss.item()))

# Generate synthetic data using the trained model
X_test = torch.randn(len(X), 8, dtype=torch.float32)
y_test = mlp(X_test).detach()

# Print the first few rows of the synthetic data
print('X1,  X2,  X3,  X4,  X5,  y')
for i in range(5):
    for j in range(5):
        print(int(X_test[i, j]), ' ',  )
        torch.round([y_test[i]])


IndexError: index 897 is out of bounds for dimension 0 with size 835